# 交叉验证

## 学习目标

- 理解交叉验证的原理和重要性
- 掌握K折交叉验证、分层交叉验证、留一法
- 学会使用交叉验证进行模型比较
- 理解不同K值对评估结果的影响

## 课程概述

本Notebook将带你全面学习交叉验证方法。交叉验证是评估模型性能的重要技术，可以充分利用数据，避免过拟合，并帮助我们选择最佳模型。

**学习路径**：
1. K折交叉验证基础
2. 分层K折交叉验证
3. 留一法（LOOCV）
4. 多指标交叉验证
5. 模型比较
6. 不同K值的影响分析


In [ ]:
# 导入必要的库
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import (cross_val_score, KFold, StratifiedKFold,
                                    LeaveOneOut, cross_validate)
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score

# 设置中文字体
plt.rcParams['font.sans-serif'] = ['SimHei']
plt.rcParams['axes.unicode_minus'] = False

# 设置随机种子
np.random.seed(42)

# 设置图表样式
plt.style.use('seaborn-v0_8-darkgrid')
print("环境准备完成！")


## 理论回顾

### 为什么需要交叉验证？

**问题**：如果只用一次训练-测试划分，评估结果可能不够稳定，因为：
- 数据划分的随机性会影响结果
- 可能过拟合测试集
- 无法充分利用数据

**解决方案**：交叉验证（Cross-Validation）

### K折交叉验证（K-Fold Cross-Validation）

**原理**：
1. 将数据分成K份（通常K=5或10）
2. 每次用K-1份训练，1份验证
3. 重复K次，得到K个评估结果
4. 计算平均值和标准差

**优点**：
- 充分利用所有数据
- 评估结果更稳定
- 可以发现过拟合问题

### 分层K折交叉验证（Stratified K-Fold）

**特点**：
- 保持每折中各类别的比例与原始数据相同
- 特别适合类别不平衡的数据

### 留一法（Leave-One-Out Cross-Validation, LOOCV）

**特点**：
- K等于样本数
- 每次只用1个样本验证
- 计算量大，但评估结果无偏


## 1. 数据准备

首先生成分类数据，用于后续的交叉验证演示。


In [ ]:
# 生成分类数据
# n_samples: 样本数量（200个样本，适合演示）
# n_features: 特征数量
# n_informative: 有信息的特征数量
# n_redundant: 冗余特征数量
# n_classes: 类别数量（二分类）
X, y = make_classification(n_samples=200, n_features=10, n_informative=5,
                          n_redundant=2, n_classes=2, random_state=42)

print("=" * 60)
print("数据信息")
print("=" * 60)
print(f"样本数: {X.shape[0]}")
print(f"特征数: {X.shape[1]}")
print(f"类别数: {len(np.unique(y))}")
print(f"类别分布: {np.bincount(y)}")


## 2. K折交叉验证

K折交叉验证是最常用的交叉验证方法。让我们看看如何使用它。


In [ ]:
# 创建模型
model = LogisticRegression(random_state=42, max_iter=1000)

# 使用5折交叉验证
# n_splits: 折数（K值）
# shuffle: 是否打乱数据
# random_state: 随机种子
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

# 进行交叉验证
# scoring: 评估指标（'accuracy'表示准确率）
scores = cross_val_score(model, X, y, cv=kfold, scoring='accuracy')

print("=" * 60)
print("K折交叉验证")
print("=" * 60)
print("\n5折交叉验证结果:")
for i, score in enumerate(scores, 1):
    print(f"  折 {i}: {score:.4f} ({score*100:.2f}%)")

print(f"\n统计结果:")
print(f"  平均准确率: {scores.mean():.4f} ({scores.mean()*100:.2f}%)")
print(f"  标准差: {scores.std():.4f}")
print(f"  最小值: {scores.min():.4f}")
print(f"  最大值: {scores.max():.4f}")

print("\n说明:")
print("- 将200个样本分成5折，每折40个样本")
print("- 每次用4折（160个样本）训练，1折（40个样本）验证")
print("- 重复5次，得到5个评估结果")
print("- 计算平均值和标准差，得到更稳定的评估")


## 3. 分层K折交叉验证

对于类别不平衡的数据，分层交叉验证可以保持每折中各类别的比例。


In [ ]:
# 使用分层5折交叉验证
# StratifiedKFold会保持每折中各类别的比例与原始数据相同
skfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scores_stratified = cross_val_score(model, X, y, cv=skfold, scoring='accuracy')

print("=" * 60)
print("分层K折交叉验证")
print("=" * 60)
print("\n分层5折交叉验证结果:")
for i, score in enumerate(scores_stratified, 1):
    print(f"  折 {i}: {score:.4f} ({score*100:.2f}%)")

print(f"\n统计结果:")
print(f"  平均准确率: {scores_stratified.mean():.4f} ({scores_stratified.mean()*100:.2f}%)")
print(f"  标准差: {scores_stratified.std():.4f}")

print("\n说明:")
print("- 分层交叉验证保持每折中各类别的比例")
print("- 对于类别不平衡的数据，分层交叉验证更准确")
print("- 通常推荐使用分层交叉验证（StratifiedKFold）")


## 4. 留一法（LOOCV）

留一法是K折交叉验证的特殊情况，K等于样本数。虽然计算量大，但评估结果无偏。


In [ ]:
# 注意：留一法计算量大，这里只用前50个样本演示
# 对于200个样本，留一法需要训练200次模型，计算时间较长
X_small = X[:50]
y_small = y[:50]

# 创建留一法交叉验证器
loocv = LeaveOneOut()

# 进行留一法交叉验证
scores_loocv = cross_val_score(model, X_small, y_small, cv=loocv, scoring='accuracy')

print("=" * 60)
print("留一法（LOOCV）")
print("=" * 60)
print(f"\n留一法交叉验证结果（前50个样本）:")
print(f"  平均准确率: {scores_loocv.mean():.4f} ({scores_loocv.mean()*100:.2f}%)")
print(f"  标准差: {scores_loocv.std():.4f}")
print(f"  评估次数: {len(scores_loocv)} (等于样本数)")

print("\n说明:")
print("- 留一法：每次只用1个样本验证，其余49个样本训练")
print("- 重复50次，得到50个评估结果")
print("- 计算量大，但评估结果无偏")
print("- 通常用于小数据集（<100个样本）")


## 5. 多指标交叉验证

使用`cross_validate`可以同时计算多个评估指标，更全面地评估模型性能。


In [ ]:
# 定义多个评估指标
# make_scorer: 将评估函数转换为scorer对象
scoring = {
    'accuracy': make_scorer(accuracy_score),
    'precision': make_scorer(precision_score),
    'recall': make_scorer(recall_score),
    'f1': make_scorer(f1_score)
}

# 使用cross_validate计算多个指标
# cross_validate返回一个字典，包含所有指标的测试分数
cv_results = cross_validate(model, X, y, cv=5, scoring=scoring)

print("=" * 60)
print("多指标交叉验证")
print("=" * 60)
print("\n多指标交叉验证结果（5折）:")
for metric in scoring.keys():
    scores = cv_results[f'test_{metric}']  # 注意：键名是 'test_指标名'
    print(f"\n{metric.upper()}:")
    print(f"  各折结果: {[f'{s:.4f}' for s in scores]}")
    print(f"  平均值: {scores.mean():.4f}")
    print(f"  标准差: {scores.std():.4f}")

print("\n说明:")
print("- cross_validate可以同时计算多个评估指标")
print("- 返回字典包含 'test_指标名' 的键")
print("- 可以更全面地评估模型性能")


## 6. 模型比较

使用交叉验证可以公平地比较不同模型的性能。


In [ ]:
# 创建多个模型
models = {
    '逻辑回归': LogisticRegression(random_state=42, max_iter=1000),
    '决策树': DecisionTreeClassifier(random_state=42),
    '随机森林': RandomForestClassifier(n_estimators=10, random_state=42)
}

# 使用交叉验证评估每个模型
results = {}
print("=" * 60)
print("模型比较")
print("=" * 60)

for name, model in models.items():
    scores = cross_val_score(model, X, y, cv=5, scoring='accuracy')
    results[name] = scores
    print(f"\n{name}:")
    print(f"  平均准确率: {scores.mean():.4f} ({scores.mean()*100:.2f}%)")
    print(f"  标准差: {scores.std():.4f}")

# 可视化模型比较
plt.figure(figsize=(10, 6))
positions = np.arange(len(models))
means = [results[name].mean() for name in models.keys()]
stds = [results[name].std() for name in models.keys()]

plt.bar(positions, means, yerr=stds, alpha=0.7, 
        color=['skyblue', 'lightgreen', 'lightcoral'],
        edgecolor='black', capsize=5, width=0.6)
plt.xticks(positions, models.keys(), fontsize=12)
plt.ylabel('准确率', fontsize=12)
plt.title('模型比较（5折交叉验证）', fontsize=14, fontweight='bold')
plt.grid(True, axis='y', alpha=0.3)

# 添加数值标签
for i, (mean, std) in enumerate(zip(means, stds)):
    plt.text(i, mean + std + 0.01, f'{mean:.3f}±{std:.3f}',
             ha='center', va='bottom', fontsize=10, fontweight='bold')

plt.tight_layout()
plt.show()

# 找出最佳模型
best_model_name = max(results.keys(), key=lambda k: results[k].mean())
print(f"\n最佳模型: {best_model_name} (准确率: {results[best_model_name].mean():.4f})")


## 7. 不同K值的影响

K值的选择会影响交叉验证的结果。让我们看看不同K值的影响。


In [ ]:
# 测试不同的K值
k_values = [3, 5, 10, 20]
k_results = {}

print("=" * 60)
print("不同K值的影响")
print("=" * 60)

for k in k_values:
    kfold = KFold(n_splits=k, shuffle=True, random_state=42)
    scores = cross_val_score(model, X, y, cv=kfold, scoring='accuracy')
    k_results[k] = scores
    print(f"\nK={k}:")
    print(f"  平均准确率: {scores.mean():.4f} ({scores.mean()*100:.2f}%)")
    print(f"  标准差: {scores.std():.4f}")
    print(f"  评估次数: {len(scores)}")

# 可视化
plt.figure(figsize=(12, 6))
means = [k_results[k].mean() for k in k_values]
stds = [k_results[k].std() for k in k_values]

plt.plot(k_values, means, marker='o', linewidth=2, markersize=10, 
         label='平均准确率', color='darkblue')
plt.fill_between(k_values, 
                 [m - s for m, s in zip(means, stds)],
                 [m + s for m, s in zip(means, stds)],
                 alpha=0.3, label='±1标准差', color='lightblue')

plt.xlabel('K值（折数）', fontsize=12)
plt.ylabel('准确率', fontsize=12)
plt.title('不同K值的交叉验证结果', fontsize=14, fontweight='bold')
plt.legend(fontsize=12)
plt.grid(True, alpha=0.3)
plt.xticks(k_values)
plt.tight_layout()
plt.show()

print("\n说明:")
print("- K值小（如3）: 计算快，但方差大，评估结果不够稳定")
print("- K值大（如20）: 计算慢，但方差小，评估结果更稳定")
print("- 通常K取5-10，平衡计算成本和评估稳定性")
print("- 对于小数据集，可以使用留一法（K=样本数）")


## 总结与思考

### 关键知识点回顾

1. **K折交叉验证**：将数据分成K份，每次用K-1份训练，1份验证，重复K次
2. **分层交叉验证**：保持每折中各类别的比例，适合类别不平衡的数据
3. **留一法**：K等于样本数，计算量大但评估结果无偏
4. **多指标交叉验证**：同时计算多个评估指标，全面评估模型性能
5. **模型比较**：使用交叉验证可以公平地比较不同模型
6. **K值选择**：通常K取5-10，平衡计算成本和评估稳定性

### 思考问题

1. **什么时候使用分层交叉验证？**
   - 类别不平衡的数据
   - 需要保持各类别比例的场景

2. **如何选择合适的K值？**
   - 小数据集（<100样本）：可以使用留一法
   - 中等数据集（100-1000样本）：K=5或10
   - 大数据集（>1000样本）：K=5即可

3. **交叉验证的优缺点？**
   - 优点：充分利用数据，评估结果稳定，可以发现过拟合
   - 缺点：计算量大，需要多次训练模型

### 下一步学习

- 学习超参数调优（GridSearchCV、RandomizedSearchCV）
- 学习特征选择方法
- 学习模型优化技巧
